In [4]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re
import json
import urllib.request

In [12]:
file_path = '/2nd_project/recipe.json'
def toJson(recipe_dict):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(recipe_dict, file, ensure_ascii=False, indent='\t')


def url_func(n,m):
    url = 'https://www.10000recipe.com/recipe/list.html?order=reco&page='
    url_list = []
    
    for num in range(n,m):
        req = urllib.request.Request(url + str(num))
        html = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(html, 'html.parser')
        
        try:
            res = soup.find(class_='cont_list st3')
            for i in res.find_all('a'):
                url_tmp = i.get('href')
                url_list.append(url_tmp)
        except(AttributeError):
            pass
    return url_list

In [ ]:
num_id = 0
food_dicts = []
url_lists = url_func(1,2)

for url_str in url_lists:
    url = 'https://www.10000recipe.com'
    url = url + url_str
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    #menu name
    res = soup.find('div', 'view2_summary')
    res = res.find('h3')
    menu_name = res.get_text().replace('\n',' ').replace('"','')

    #menu image
    res = soup.find('div', 'centeredcrop')
    res = res.find('img')
    menu_img = res.get('src')

    # ingredients
    res = soup.find('div','ready_ingre3')
    ingre_dict = {}
    ingre_list = []
    try:
        for n in res.find_all('ul'):
            for tmp in n.find_all('li'):
                ingredient_name = tmp.get_text().replace('\n','').replace(' ','')
                count = tmp.find('span')
                ingredient_tmp = count.get_text()
                ingredient_name = re.sub(ingredient_tmp, '',ingredient_name)
                ingredient_unit = ingredient_tmp.replace('/','').replace('+','')
                ingredient_unit = ''.join([i for i in ingredient_unit if not i.isdigit()])
                ingredient_count = re.sub(ingredient_unit, '', ingredient_tmp)

                ingre_dict = {"ingre_name": ingredient_name,
                             "ingre_count": ingredient_count,
                             "ingre_unit": ingredient_unit}

                ingre_list.append(ingre_dict)
    except(AttributeError):
        pass

    # recipe
    recipe_list = []
    res = soup.find('div','view_step')
    ind = 1
    for n in res.find_all('div', 'view_step_cont'):
        recipe_step_txt = n.get_text().replace('\n',' ')
        recipe_list.append(str(ind) + '. ' + recipe_step_txt)
        ind += 1

    food_dict = {"id":num_id,
                "name":menu_name,
                "img":menu_img,
                "ingre":ingre_list,
                "reci":recipe_list}
    num_id += 1
    food_dicts.append(food_dict)
    print(food_dict)

toJson(food_dicts)

In [9]:
url = 'https://www.10000recipe.com/recipe/6854292'
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
    
#menu name
res = soup.find('div', 'view2_summary')
res = res.find('h3')
menu_name = res.get_text().replace('\n',' ').replace('"','')

#menu image
res = soup.find('div', 'centeredcrop')
res = res.find('img')
menu_img = res.get('src')

# ingredients
res = soup.find('div','ready_ingre3')
ingre_dict = {}
ingre_list = []
try:
    for n in res.find_all('ul'):
        for tmp in n.find_all('li'):
            ingredient_name = tmp.get_text().replace('\n','').replace(' ','')
            count = tmp.find('span')
            ingredient_tmp = count.get_text()
            ingredient_name = re.sub(ingredient_tmp, '',ingredient_name)
            ingre_list.append(ingredient_name)
except(AttributeError):
    pass

res = soup.find('div','view_step')
print(res)


food_dict = {"id":0,
            "name":menu_name,
            "thumbnail":menu_img,
            "url":url,
            "content":res}
# print(food_dict)

<div class="view_step">
<div class="best_tit">
<b>조리순서</b><span>Steps</span>
<div class="best_tit_rmn">
<a href="javascript:void(0);" onclick="chgViewStep(1)"><img alt="이미지크게보기" id="tabStepView1" src="https://recipe1.ezmember.co.kr/img/mobile/tab_view1.png"/></a>
<a href="javascript:void(0);" onclick="chgViewStep(2)"><img alt="텍스트만보기" id="tabStepView2" src="https://recipe1.ezmember.co.kr/img/mobile/tab_view2.png"/></a>
<a href="javascript:void(0);" onclick="chgViewStep(3)"><img alt="이미지작게보기" id="tabStepView3" src="https://recipe1.ezmember.co.kr/img/mobile/tab_view3_on.png"/></a>
<button class="btn btn-sm btn-default" onclick="location.href='/recipe/6854292?seq=6854292&amp;is_origin=y'" style="margin-left:10px;">원본보기</button>
</div>
</div>
<div class="view_step_cont media step1" id="stepDiv1"><div class="media-body" id="stepdescr1">떡은 물에 행궈서 한번 데쳐줘요~<br>
그래야 떡이 좀더 야들야들 하거든요~</br></div><div id="stepimg1"><img src="https://recipe1.ezmember.co.kr/cache/recipe/2016/08/09/3fde1193e9c84475760